In [ ]:
import geopandas as gpd

# Define the path to the shapefile
shapefile_path = "data/ESRI IN Village/India%3A_Village_Boundary_2021/India%3A_Village_Boundary_2021.shp"

# Read the shapefile using geopandas
shapefile = gpd.read_file(shapefile_path)

# Print the column names
print(shapefile.columns)

In [ ]:
import rasterio

# Define the path to the raster file
raster_file = "data/IA2020DHS_AHTOBCMANY_MS_v01/IA2020DHS_AHTOBCMANY_MS_MEAN_v01.tif"

# Open the raster file
with rasterio.open(raster_file) as src:
    # Get the metadata of the raster
    meta = src.meta

# Print the metadata
print(meta)

In [ ]:
import rasterio
import geopandas as gpd

# Load the shapefile
villages = gpd.read_file('data/ESRI IN Village/India%3A_Village_Boundary_2021/India%3A_Village_Boundary_2021.shp')

# Load the raster file
raster = rasterio.open('data/IA2020DHS_AHTOBCMANY_MS_v01/IA2020DHS_AHTOBCMANY_MS_CI_v01.tif')


In [ ]:
print(villages.crs)
print(raster.crs)
from rasterstats import zonal_stats

stats = zonal_stats(villages, 'data/IA2020DHS_AHTOBCMANY_MS_v01/IA2020DHS_AHTOBCMANY_MS_CI_v01.tif', stats="mean", nodata=raster.nodata)


In [ ]:
pip install rasterstats

In [ ]:
import pandas as pd

# Convert the list of dictionaries to a DataFrame
stats_df = pd.DataFrame(stats)

# Merge the statistics with the original shapefile
villages_with_stats = pd.concat([villages, stats_df], axis=1)


In [ ]:
villages_with_stats.to_file('data/ESRI IN Village/India%3A_Village_Boundary_2021/NHS20_AH_TOBC_Village_Boundary_2021.shp')


In [ ]:
muzaffarnagar_data = villages_with_stats[
    (villages_with_stats['district'] == 'Muzaffarnagar') & 
    (villages_with_stats['state'] == 'Uttar Pradesh')
]


In [ ]:
pip install folium
import folium

# Create a base map centered around Muzaffarnagar
m = folium.Map(location=[29.4739, 77.7041], zoom_start=10)

# Add the village data to the map
folium.Choropleth(
    geo_data=muzaffarnagar_data,
    data=muzaffarnagar_data,
    columns=['id', 'mean'],  # Assuming 'id' is a unique identifier for each village; replace if different
    key_on='feature.properties.id',
    fill_color='YlGn',
    legend_name='Mean Raster Value'
).add_to(m)

# Display the map
m


In [ ]:
import matplotlib.pyplot as plt

# Descriptive statistics
print(muzaffarnagar_data['mean'].describe())

# Histogram
muzaffarnagar_data['mean'].hist(bins=30, edgecolor='black')
plt.title('Distribution of Mean Raster Values')
plt.xlabel('Mean Raster Value')
plt.ylabel('Number of Villages')
plt.show()


In [ ]:
import folium

# Create a base map centered around Muzaffarnagar
m = folium.Map(location=[29.4739, 77.7041], zoom_start=10)

# Define a function to assign colors based on the custom classes
def custom_color(value):
    if value < 0.084:
        return '#2c7bb6'  # Color for Extremely Low (Blue)
    elif value < 0.100:
        return '#abd9e9'  # Color for Low (Light Blue)
    elif value < 0.116:
        return '#ffffbf'  # Color for Medium-Low (Yellow)
    elif value < 0.136:
        return '#fdae61'  # Color for Medium-High (Orange)
    else:
        return '#d7191c'  # Color for High (Red)

# Add the village data to the map with custom colors
for _, row in muzaffarnagar_data.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=lambda feature, row=row: {
            'fillColor': custom_color(row['mean']),
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        }
    ).add_to(m)

# Add a custom legend
legend_html = """
<div style="position: fixed; bottom: 50px; left: 50px; width: 150px; height: 150px; background-color: white; border:2px solid grey; z-index:9999; font-size:14px;">
    &nbsp;<b>Legend</b><br>
    &nbsp;<i class="fa fa-square fa-2x" style="color:#2c7bb6"></i>&nbsp;0.000 - 0.084<br>
    &nbsp;<i class="fa fa-square fa-2x" style="color:#abd9e9"></i>&nbsp;0.084 - 0.100<br>
    &nbsp;<i class="fa fa-square fa-2x" style="color:#ffffbf"></i>&nbsp;0.100 - 0.116<br>
    &nbsp;<i class="fa fa-square fa-2x" style="color:#fdae61"></i>&nbsp;0.116 - 0.136<br>
    &nbsp;<i class="fa fa-square fa-2x" style="color:#d7191c"></i>&nbsp;0.136 - 0.163<br>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# Display the map
m
